# Sales Transactions Weekly Time Series

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('../data/Sales_Transactions_Dataset_Weekly.csv')
df.head(5)

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,Normalized 42,Normalized 43,Normalized 44,Normalized 45,Normalized 46,Normalized 47,Normalized 48,Normalized 49,Normalized 50,Normalized 51
0,P1,11,12,10,8,13,12,14,21,6,...,0.06,0.22,0.28,0.39,0.50,0.00,0.22,0.17,0.11,0.39
1,P2,7,6,3,2,7,1,6,3,3,...,0.20,0.40,0.50,0.10,0.10,0.40,0.50,0.10,0.60,0.00
2,P3,7,11,8,9,10,8,7,13,12,...,0.27,1.00,0.18,0.18,0.36,0.45,1.00,0.45,0.45,0.36
3,P4,12,8,13,5,9,6,9,13,13,...,0.41,0.47,0.06,0.12,0.24,0.35,0.71,0.35,0.29,0.35
4,P5,8,5,13,11,6,7,9,14,9,...,0.27,0.53,0.27,0.60,0.20,0.20,0.13,0.53,0.33,0.40


In [3]:
df = df.filter(regex=r'Product|W')
df.head(5)

,Product_Code,W0,W1,W2,W3,W4,W5,W6,W7,W8,...,W42,W43,W44,W45,W46,W47,W48,W49,W50,W51
0,P1,11,12,10,8,13,12,14,21,6,...,4,7,8,10,12,3,7,6,5,10
1,P2,7,6,3,2,7,1,6,3,3,...,2,4,5,1,1,4,5,1,6,0
2,P3,7,11,8,9,10,8,7,13,12,...,6,14,5,5,7,8,14,8,8,7
3,P4,12,8,13,5,9,6,9,13,13,...,9,10,3,4,6,8,14,8,7,8
4,P5,8,5,13,11,6,7,9,14,9,...,7,11,7,12,6,6,5,11,8,9


In [4]:
melt = df.melt(id_vars='Product_Code', var_name='Week', value_name='Sales')
melt.head()

,Product_Code,Week,Sales
0,P1,W0,11
1,P2,W0,7
2,P3,W0,7
3,P4,W0,12
4,P5,W0,8


In [5]:
melt['Product_Code'] = melt['Product_Code'].str.extract('(\d+)', expand=False).astype(int)
melt['Week'] = melt['Week'].str.extract('(\d+)', expand=False).astype(int)

melt = melt.sort_values(['Week','Product_Code'])
melt.head()

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7
2,3,0,7
3,4,0,12
4,5,0,8


# Split the data

In [6]:
split_point = 40
melt_train = melt[melt['Week'] < split_point].copy()
melt_valid = melt[melt['Week'] >= split_point].copy()

In [7]:
melt_train.head(2)

,Product_Code,Week,Sales
0,1,0,11
1,2,0,7


In [8]:
melt_train.tail(2)

,Product_Code,Week,Sales
32438,818,39,0
32439,819,39,0


In [9]:
melt_valid.head(2)

,Product_Code,Week,Sales
32440,1,40,7
32441,2,40,2


In [10]:
melt_valid.tail(2)

,Product_Code,Week,Sales
42170,818,51,0
42171,819,51,1


# Set up a 1-step target

In [11]:
melt_train['sales_next_week'] = melt_train.groupby('Product_Code')['Sales'].shift(-1)

In [12]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week
0,1,0,11,12.0
811,1,1,12,10.0
1622,1,2,10,8.0
2433,1,3,8,13.0
3244,1,4,13,12.0


In [13]:
melt_valid['sales_next_week'] = melt_valid.groupby('Product_Code')['Sales'].shift(-1)

In [14]:
melt_valid[melt_valid['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week
32440,1,40,7,11.0
33251,1,41,11,4.0
34062,1,42,4,7.0
34873,1,43,7,8.0
35684,1,44,8,10.0


In [15]:
melt_train = melt_train.dropna()
melt_valid = melt_valid.dropna()

In [16]:
melt_train.tail()

,Product_Code,Week,Sales,sales_next_week
31624,815,38,1,2.0
31625,816,38,4,6.0
31626,817,38,0,0.0
31627,818,38,0,0.0
31628,819,38,1,0.0


In [17]:
melt_valid.tail()

,Product_Code,Week,Sales,sales_next_week
41356,815,50,2,0.0
41357,816,50,6,5.0
41358,817,50,4,3.0
41359,818,50,2,0.0
41360,819,50,0,1.0


# Create Fundamental Features:

- LAG
- Difference
- Statistics: Mean / MAX / MIN / Std
- Data Components (seasonality):Day of week \ Day of year \ Month \ Day of month
- Interactions among data components. e.g the sales of this week minus the average for week sales value. Rooling avg for week divided by std deviavion for the same week. 

In [18]:
# Lag what if I dont have the last period? get the closest available
# LAG = Last values
melt_train['lag_sales_1'] = melt_train.groupby('Product_Code')['Sales'].shift(1) #last row will align with the next row
melt_train[melt_train['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
0,1,0,11,12.0,NaN
811,1,1,12,10.0,11.0
1622,1,2,10,8.0,12.0
2433,1,3,8,13.0,10.0
3244,1,4,13,12.0,8.0


In [19]:
melt_valid['lag_sales_1'] = melt_valid.groupby('Product_Code')['Sales'].shift(1) #last row will align with the next row
melt_valid[melt_valid['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1
32440,1,40,7,11.0,NaN
33251,1,41,11,4.0,7.0
34062,1,42,4,7.0,11.0
34873,1,43,7,8.0,4.0
35684,1,44,8,10.0,7.0


In [20]:
# Difference
melt_train['diff_sales_1'] = melt_train.groupby('Product_Code')['Sales'].diff(1)
melt_train[melt_train['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
0,1,0,11,12.0,NaN,NaN
811,1,1,12,10.0,11.0,1.0
1622,1,2,10,8.0,12.0,-2.0
2433,1,3,8,13.0,10.0,-2.0
3244,1,4,13,12.0,8.0,5.0


In [21]:
# Difference
melt_valid['diff_sales_1'] = melt_valid.groupby('Product_Code')['Sales'].diff(1)
melt_valid[melt_valid['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1
32440,1,40,7,11.0,NaN,NaN
33251,1,41,11,4.0,7.0,4.0
34062,1,42,4,7.0,11.0,-7.0
34873,1,43,7,8.0,4.0,3.0
35684,1,44,8,10.0,7.0,1.0


In [22]:
# Rolling Statistics
melt_train['mean_sales_4'] = melt_train.groupby('Product_Code')['Sales'].rolling(4).mean().reset_index(level=0,drop=True)
melt_train[melt_train['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
811,1,1,12,10.0,11.0,1.0,NaN
1622,1,2,10,8.0,12.0,-2.0,NaN
2433,1,3,8,13.0,10.0,-2.0,10.25
3244,1,4,13,12.0,8.0,5.0,10.75


In [23]:
melt_valid['mean_sales_4'] = melt_valid.groupby('Product_Code')['Sales'].rolling(4).mean().reset_index(level=0,drop=True)
melt_valid[melt_valid['Product_Code'] == 1].head() 

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
32440,1,40,7,11.0,NaN,NaN,NaN
33251,1,41,11,4.0,7.0,4.0,NaN
34062,1,42,4,7.0,11.0,-7.0,NaN
34873,1,43,7,8.0,4.0,3.0,7.25
35684,1,44,8,10.0,7.0,1.0,7.50


#Data Components (seasonality):
-Day of week
-Day of year
-Month
-Day of month

# MAPE mean absolute percentage error
No accept zeros

# WMAPE weighted mean absolute percentage error
allows division by zero

In [24]:
def mape(y_true, y_pred):
    ape = np.abs((y_true - y_pred) / y_true)
    ape[~np.isfinite(ape)] = 1.
    return np.mean(ape)

def wmape(y_true, y_pred):
    return np.sum(np.abs(y_true - y_pred)) / np.sum(np.abs(y_true))

# Establish baseline
Predict next week sales as equal to ths week sales
tomorrow's resulta is equal to today's result.

In [25]:
y_pred = melt_train['Sales']
y_true = melt_train['sales_next_week']

In [26]:
mape(y_true, y_pred)

0.6721872645511404

In [27]:
wmape(y_true, y_pred)

0.30816465612331645

# Train the Model

In [28]:
melt_train.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4
0,1,0,11,12.0,NaN,NaN,NaN
1,2,0,7,6.0,NaN,NaN,NaN
2,3,0,7,11.0,NaN,NaN,NaN
3,4,0,12,8.0,NaN,NaN,NaN
4,5,0,8,5.0,NaN,NaN,NaN


In [29]:
features = ['Sales','lag_sales_1','diff_sales_1','mean_sales_4']

In [30]:
#Avg of each columns and will replace by average of the columns
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train['sales_next_week']

mdl = RandomForestRegressor(n_estimators=100,random_state=0,n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

# Evaluate the model

In [31]:
#imputer to handle missing values
Xval = imputer.transform(melt_valid[features])
yval = melt_valid['sales_next_week']

p = mdl.predict(Xval)

In [32]:
mape(yval,p)

0.6141604503405896

In [33]:
wmape(yval,p)

0.3004969729507602

# Extend the model to predict n-steps

In [34]:
melt_train['sales_next_next_week'] = melt_train.groupby('Product_Code')['Sales'].shift(-2)
melt_valid['sales_next_next_week'] = melt_valid.groupby('Product_Code')['Sales'].shift(-2)

In [35]:
melt_train[melt_train['Product_Code'] == 1].head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
0,1,0,11,12.0,NaN,NaN,NaN,10.0
811,1,1,12,10.0,11.0,1.0,NaN,8.0
1622,1,2,10,8.0,12.0,-2.0,NaN,13.0
2433,1,3,8,13.0,10.0,-2.0,10.25,12.0
3244,1,4,13,12.0,8.0,5.0,10.75,14.0


In [36]:
melt_train = melt_train.dropna(subset=['sales_next_week','sales_next_next_week'])

In [37]:
#Avg of each columns and will replace by average of the columns
imputer = SimpleImputer()
Xtr = imputer.fit_transform(melt_train[features])
ytr = melt_train[['sales_next_week','sales_next_next_week']]

mdl = RandomForestRegressor(n_estimators=100,random_state=0,n_jobs=6)
mdl.fit(Xtr, ytr)

RandomForestRegressor(n_jobs=6, random_state=0)

In [38]:
#imputer to handle missing values
Xval = imputer.transform(melt_valid[features])
yval = melt_valid[['sales_next_week','sales_next_next_week']]

p = mdl.predict(Xval)

In [39]:
mape(yval,p)

0.6528889309606773

In [40]:
wmape(yval,p)

sales_next_week         0.300301
sales_next_next_week    0.298772
dtype: float64

# Predicting new examples - Using new data
Since you have the same features

In [41]:
melt_valid.tail()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
41356,815,50,2,0.0,0.0,2.0,0.75,NaN
41357,816,50,6,5.0,5.0,1.0,5.25,NaN
41358,817,50,4,3.0,0.0,4.0,1.00,NaN
41359,818,50,2,0.0,0.0,2.0,0.50,NaN
41360,819,50,0,1.0,0.0,0.0,0.00,NaN


In [47]:
new_data = melt_valid[melt_valid['Week'] == 50].copy()
new_data.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week
40550,1,50,5,10.0,6.0,-1.0,5.25,NaN
40551,2,50,6,0.0,1.0,5.0,4.00,NaN
40552,3,50,8,7.0,8.0,0.0,9.50,NaN
40553,4,50,7,8.0,8.0,-1.0,9.25,NaN
40554,5,50,8,9.0,11.0,-3.0,7.50,NaN


In [49]:
predictions = mdl.predict(new_data[features])
predictions

C:\Users\User\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


array([[4.56808081, 5.47370094],
       [3.02466667, 4.414     ],
       [8.13699634, 6.93155861],
       ...,
       [0.36393611, 0.65634303],
       [0.63814973, 0.5276208 ],
       [0.15293679, 0.15757401]])

In [51]:
new_data['prediction_sales_next_week'] = predictions[:,0]
new_data['prediction_sales_next_week'] = predictions[:,1]

In [52]:
new_data.head()

,Product_Code,Week,Sales,sales_next_week,lag_sales_1,diff_sales_1,mean_sales_4,sales_next_next_week,prediction_sales_next_week
40550,1,50,5,10.0,6.0,-1.0,5.25,NaN,5.473701
40551,2,50,6,0.0,1.0,5.0,4.00,NaN,4.414000
40552,3,50,8,7.0,8.0,0.0,9.50,NaN,6.931559
40553,4,50,7,8.0,8.0,-1.0,9.25,NaN,5.801071
40554,5,50,8,9.0,11.0,-3.0,7.50,NaN,7.306667


# Next Steps:
- more features with more periods
- different models
- find which products have the worst errors and check if there is afixable pattern
- tune the model
- Which are the products with the worse MAPE? So, find errors. 

In [ ]:
- 